In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from lxml import html
import pandas as pd
import openpyxl
import unicodedata
from mplsoccer import PyPizza, FontManager, add_image
from PIL import Image, ImageDraw, ImageOps
import matplotlib.pyplot as plt
import time
import requests

In [11]:
def getReports():

#open the url and extract the html and write it to a new file
    #html = urlopen('https://fbref.com/en/comps/9/2023-2024/stats/2023-2024-Premier-League-Stats')
    url = 'https://fbref.com/en/comps/9/2023-2024/stats/2023-2024-Premier-League-Stats'
    # Set headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    # Send the request
    response = requests.get(url, headers=headers)
    bs = BeautifulSoup(response.content, 'html.parser').encode()
    return bs

getReports()

b'<!DOCTYPE html>\n<html lang="en-US"><head><title>Just a moment...</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="noindex,nofollow" name="robots"/><meta content="width=device-width,initial-scale=1" name="viewport"/><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}@media (prefers-color-scheme:dark){body{background-color:#222;color:#d9d9d9}body a{color:#fff}body a:hover{color:#ee730a;text-decoration:underline}body .lds-ring div{border-color:#999 transparent transparent}body .font-red{color:#b20f03}body .pow-button{background-color:#4693ff;color:#1d1d1d}body #challenge-success-text{background-image:url()}body #challenge-error-text{backgrou

In [9]:
def getReports():

#open the url and extract the html and write it to a new file
    #html = urlopen('https://fbref.com/en/comps/9/2023-2024/stats/2023-2024-Premier-League-Stats')
    url = 'https://fbref.com/en/comps/9/2023-2024/stats/2023-2024-Premier-League-Stats'
    # Set headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    # Send the request
    response = requests.get(url, headers=headers)
    bs = BeautifulSoup(response, 'html.parser').encode()

    with open("data1.html", "w") as file:
        file.write(str(bs))
        
#open the html file and replace the comment part with void so that all the tables can now be accessed
    with open("data1.html",'r') as html1:
        bs2 = BeautifulSoup(html1, 'html.parser')
        text_of_bs2 = str(bs2)
        usable_bs2 = text_of_bs2.replace("<!--"," ").replace("-->", " ")
        with open("data2.html", 'w') as commentless_content:
            commentless_content.write(usable_bs2)

#create an excel sheet and add links to player profiles so as if fetch it if need be
    with open('data2.html', 'r') as main_page:
        bs3 = BeautifulSoup(main_page,'html.parser')
        table_contents = bs3.find_all('table')
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet['A1'] = "Name"
        sheet['B1'] = "Link"
        workbook.save('player_profiles.xlsx')
        for count, content in enumerate(table_contents):
            list_of_links = content.find_all('a', href= re.compile(r"^/en/players/[a-f0-9]{8}/[A-Za-z-]+$"))
            for link in list_of_links:
                if 'href' in link.attrs:
                        locA = "A"+str(count)
                        locB = "B"+str(count)
                        sheet[locA] = link.text
                        sheet[locB] = link.attrs['href']
                        workbook.save('player_profiles.xlsx')
getReports()

#since some player have non-english letters in their names like 'ø' for 'o' etc.,
#this code replaces such letters with corresponding english letters 
def name_updater():
    count = 1
    df = pd.read_excel('player_profiles.xlsx')
    while count < df.shape[0]:
        real_name = str(df['Link'].values[count]).split('/').pop().replace('-'," ")
        df['Name'].values[count] = real_name
        df.to_excel('player_profiles.xlsx')
        count+=1
name_updater()

TypeError: object of type 'Response' has no len()